# Model

### package load

In [1]:
import datetime
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
import time
from tqdm import tqdm
import numpy as np
import warnings
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch.autograd import Variable

from model_new import convert_bn_to_instancenorm, convert_bn_to_evonorm, convert_bn_to_groupnorm, DeepLabHead 
from helpers import AverageMeter, ProgressMeter, iouCalc, visim, vislbl
from labels import labels

### device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### data load

In [3]:
npzfile = np.load('data_norm_v2.npz') ## or 'data_origin.npz' 원본 or norm ver :: 모델 결과 확인 후 결정

train_x = npzfile['train_x']
train_y = npzfile['train_y']
test_x = npzfile['test_x']
test_y = npzfile['test_y']

npzfile.close()

### DeepLab v3 ResNet50 

In [31]:
train_x.shape

(560, 720, 960, 3)

In [4]:
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False).to(device)
         # dlv3 backbone resnoet 50 :: output chnnel: 2048 --> DL input channel :: 2048
model.classifier = DeepLabHead(2048, 12).to(device) # 12 = class num
   # 작동되는 지 확인 필요 (개수 맞추기)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2) 

In [5]:
# Initialize metrics
best_miou = 0.0
metrics = {'train_loss' : [],
           'train_acc' : [],
           'test_acc' : [],
           'test_loss' : [],
           'miou' : []}
start_epoch = 0

### Label 

In [6]:
# Create list of class names
  # classLabels: label명
  # validClasses: label에 해당하는 id값

classLabels = []
for label in labels:
    if label.name not in classLabels:
        classLabels.append(label.name)
classLabels.append('void')

validClasses = list(
    np.unique(
        [label.id for label in labels 
         if label.id >= 0
        ] + [11]
    )
)

### train

In [7]:
## parameter

batch_size = 2
num_epoch = 5

result_dir = os.getcwd() + '/result/' + 'new/'

In [8]:
from torch.utils.data import TensorDataset, DataLoader

   # X: 0~255, RGB값 가지는 이미지 데이터 (560, 720, 960, 3)
   # Y: 0~11 label값 가지는 픽셀당 클래스 데이터 (560, 720, 960)
X = torch.tensor(train_x, dtype=torch.float32)
Y = torch.tensor(train_y, dtype=torch.long)

data = TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)
train_data = DataLoader(data, batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss(weight = None, ignore_index=12)

In [9]:
model.train()
res = X.shape[1] * X.shape[2] # 720 * 960

for epoch in range(0, num_epoch):
    
    loss_running = AverageMeter('Loss', ':.4e')
    acc_running = AverageMeter('Accuracy', ':.3f')
    
    iou = iouCalc(classLabels, validClasses, voidClass = 11)
    progress = ProgressMeter(
        len(train_data),
        [loss_running, acc_running],
        prefix="Train, epoch: [{}]".format(epoch))
    
    batch_loss = 0.0
    for batch, (x, y) in enumerate(tqdm(train_data, total=len(train_data))):
        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(x)
        outputs = outputs['out']
        preds = torch.argmax(outputs, 1)
        
        # cross-entropy loss
        loss = criterion(outputs, y)

        # backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        bs = x.size(0)
        loss = loss.item()
        loss_running.update(loss, bs)
        corrects = torch.sum((preds == y) & (y != 12))
        
        nvoid = int((y==12).sum()) 
        acc = corrects.double()/(bs*res-nvoid)
        acc_running.update(acc, bs)
        
        # Calculate IoU scores of current batch
        iou.evaluateBatch(preds, y)
        
        progress.display(epoch)
        
        
    scheduler.step(loss_running.avg)
    miou = iou.outputScores()
    
    # save checkpoint
    now = datetime.datetime.now()
    now_time = now.strftime('%y%m%d%H')
    
    # save path
    if not os.path.isdir(result_dir):
        os.makedirs(result_dir)
    
    save_path = result_dir
    
    with open(save_path + 'train_logs.csv', 'a') as epochs:
            epochs.write('{}, {:.4f}, {:.4f}, {:.4f}\n'.format(
                    epoch+1, loss_running.avg, acc_running.avg, miou))
            
    # Save model to file
    torch.save({
        'epoch' : epoch + 1,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_miou': best_miou,
        'metrics': metrics,
        }, save_path + now_time + '_E' + str(epoch+1) + '_checkpoint.pth.tar')
    
    # Save best model to file
    if miou > best_miou:
        print('mIoU improved from {:.4f} to {:.4f}.'.format(best_miou, miou))
        best_miou = miou
        
    print('epoch ', epoch)
    print('loss : {:.4f}   acc : {:.4f}   miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

  0%|▎                                                                               | 1/280 [00:17<1:19:54, 17.19s/it]

Train, epoch: [0][  0/280]	Loss 2.4789e+00 (2.4789e+00)	Accuracy 0.167 (0.167)


  1%|▌                                                                               | 2/280 [00:34<1:20:08, 17.30s/it]

Train, epoch: [0][  0/280]	Loss 2.2859e+00 (2.3824e+00)	Accuracy 0.270 (0.219)


  1%|▊                                                                               | 3/280 [00:51<1:19:18, 17.18s/it]

...

100%|█████████████████████████████████████████████████████████████████████████████▋| 279/280 [1:19:27<00:16, 16.53s/it]

Train, epoch: [4][  4/280]	Loss 2.1078e-01 (1.8381e-01)	Accuracy 0.926 (0.936)


100%|██████████████████████████████████████████████████████████████████████████████| 280/280 [1:19:44<00:00, 17.09s/it]

Train, epoch: [4][  4/280]	Loss 1.4377e-01 (1.8366e-01)	Accuracy 0.952 (0.936)
classes           IoU
---------------------
Pole          : 0.190
SignSymbol    : 0.645
Bicyclist     : 0.787
Pedestrian    : 0.595
Building      : 0.911
Fence         : 0.790
Pavement      : 0.881
Road          : 0.974
Car           : 0.887
Sky           : 0.920
Tree          : 0.851
---------------------
Mean IoU      : 0.766
---------------------


mIoU improved from 0.7482 to 0.7664.
epoch  4
loss : 0.1837   acc : 0.9363   miou : 0.7664


### test

In [10]:
X = torch.tensor(test_x, dtype=torch.float32)
Y = torch.tensor(test_y, dtype=torch.long)

data = torch.utils.data.TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)

test_data = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

save_path = result_dir
result = sorted(os.listdir(save_path), reverse=True)

In [21]:
result

['val_logs.csv',
 'train_logs.csv',
 '21060800_E2_checkpoint.pth.tar',
 '21060722_E1_checkpoint.pth.tar']

In [11]:
checkpoint = torch.load(save_path + result[1]) # train, val 결과보고 결정 (overfitting 시작되기 전)
model.load_state_dict(checkpoint['model_state_dict'], strict=True)

<All keys matched successfully>

In [12]:
batch_time = AverageMeter('Time', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
progress = ProgressMeter(
    len(test_data),
    [batch_time, data_time],
    prefix='Predict: ')
loss_running = AverageMeter('Loss', ':.4e')
acc_running = AverageMeter('Accuracy', ':.3f')
    
iou = iouCalc(classLabels, validClasses, voidClass = 11)

model.eval()

batch_loss = 0.0
for batch, (x, y) in enumerate(tqdm(test_data, total=len(test_data))):

    x = x.to(device)
    y = y.to(device)

    # forward
    outputs = model(x)
    outputs = outputs['out']

    preds = torch.argmax(outputs, 1)

    # cross-entropy loss
    loss = criterion(outputs, y)

    # Statistics
    bs = x.size(0)
    loss = loss.item()
    loss_running.update(loss, bs)
    corrects = torch.sum((preds == y) & (y != 12))

    nvoid = int((y==12).sum())
    acc = corrects.double()/(bs*res-nvoid)
    acc_running.update(acc, bs)

    # Calculate IoU scores of current batch
    iou.evaluateBatch(preds, y)

miou = iou.outputScores()

print('loss : {:.4f} acc : {:.4f} miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

100%|██████████████████████████████████████████████████████████████████████████████████| 71/71 [07:15<00:00,  6.13s/it]

classes           IoU
---------------------
Pole          : 0.173
SignSymbol    : 0.539
Bicyclist     : 0.736
Pedestrian    : 0.528
Building      : 0.870
Fence         : 0.411
Pavement      : 0.808
Road          : 0.959
Car           : 0.834
Sky           : 0.917
Tree          : 0.788
---------------------
Mean IoU      : 0.688
---------------------
loss : 0.2840 acc : 0.9075 miou : 0.6876
